## Import

In [1]:
import warnings

warnings.filterwarnings('ignore')

In [103]:
import pickle
import random as rnd
import warnings

import matplotlib.pyplot as plt
import numpy as np
import pandas as pd
import seaborn as sn
import seaborn as sns
import sklearn
import statsmodels.api as sm
import xgboost as xgb
from matplotlib.font_manager import FontProperties
from sklearn import metrics
from sklearn.decomposition import PCA
from sklearn.discriminant_analysis import (LinearDiscriminantAnalysis,
                                           QuadraticDiscriminantAnalysis)
from sklearn.ensemble import (AdaBoostRegressor, BaggingClassifier,
                              BaggingRegressor, ExtraTreesRegressor,
                              GradientBoostingRegressor, RandomForestRegressor,
                              VotingRegressor)
from sklearn.feature_selection import (RFE, SelectFromModel, SelectKBest,
                                       VarianceThreshold, chi2)
from sklearn.inspection import permutation_importance
from sklearn.linear_model import (ElasticNet, HuberRegressor, Lasso,
                                  LinearRegression, RANSACRegressor, Ridge,
                                  SGDRegressor, TheilSenRegressor)
from sklearn.metrics import (accuracy_score, balanced_accuracy_score,
                             classification_report, confusion_matrix, log_loss,
                             precision_score, recall_score, roc_auc_score, mean_squared_error, r2_score)
from sklearn.model_selection import (GridSearchCV, KFold, RandomizedSearchCV,
                                     ShuffleSplit, StratifiedKFold,
                                     cross_val_score, cross_validate,
                                     train_test_split)
from sklearn.neighbors import KNeighborsRegressor
from sklearn.pipeline import Pipeline
from sklearn.preprocessing import (Binarizer, LabelEncoder, MinMaxScaler,
                                   Normalizer, OneHotEncoder,
                                   PolynomialFeatures, StandardScaler,
                                   normalize)
from sklearn.svm import SVR, LinearSVR
from sklearn.tree import DecisionTreeRegressor
from xgboost import XGBRegressor

pd.set_option('display.max_rows', 500)
pd.set_option('display.max_columns', 500)

## Reading data:

datapath : "../data"

In [104]:
traindf = pd.read_csv('../data/train.csv', index_col="ID")
traindf.sample(2)

,hotel,is_canceled,lead_time,arrival_date_year,arrival_date_month,arrival_date_week_number,arrival_date_day_of_month,stays_in_weekend_nights,stays_in_week_nights,adults,children,babies,meal,country,market_segment,distribution_channel,is_repeated_guest,previous_cancellations,previous_bookings_not_canceled,reserved_room_type,assigned_room_type,booking_changes,deposit_type,agent,company,days_in_waiting_list,customer_type,adr,required_car_parking_spaces,total_of_special_requests,reservation_status,reservation_status_date
ID,,,,,,,,,,,,,,,,,,,,,,,,,,,,,,,,
90402,City Hotel,1,111,2017,March,12,24,1,2,2,0.0,0,BB,POL,Online TA,TA/TO,0,0,0,D,D,0,No Deposit,9.0,NaN,0,Transient,97.375337,0,1,Canceled,2016-12-05
13237,City Hotel,1,59,2015,October,41,8,2,3,2,0.0,0,BB,PRT,Online TA,TA/TO,0,0,0,A,A,0,No Deposit,9.0,NaN,0,Contract,88.856641,0,2,Canceled,2015-09-24


### Drop columns not in test data and Clean missing data
* Drop columns: `is_canceled`, `reservation_status`, `reservation_status_date`, where they aren't in test data

In [105]:
testCol = set(pd.read_csv('../data/test.csv', index_col="ID").columns)
removeCols = [col for col in traindf.columns if col not in testCol]
removeCols.remove("adr")
print(removeCols)

['is_canceled', 'reservation_status', 'reservation_status_date']


In [106]:
traindf = traindf.drop(removeCols, axis=1)
traindf.sample(2)

,hotel,lead_time,arrival_date_year,arrival_date_month,arrival_date_week_number,arrival_date_day_of_month,stays_in_weekend_nights,stays_in_week_nights,adults,children,babies,meal,country,market_segment,distribution_channel,is_repeated_guest,previous_cancellations,previous_bookings_not_canceled,reserved_room_type,assigned_room_type,booking_changes,deposit_type,agent,company,days_in_waiting_list,customer_type,adr,required_car_parking_spaces,total_of_special_requests
ID,,,,,,,,,,,,,,,,,,,,,,,,,,,,,
47486,Resort Hotel,304,2016,June,26,20,4,10,2,0.0,0,BB,GBR,Offline TA/TO,TA/TO,0,0,0,D,D,0,No Deposit,40.0,NaN,0,Contract,12.352851,0,1
70680,Resort Hotel,41,2016,November,45,2,3,9,2,0.0,0,BB,PRT,Online TA,TA/TO,0,0,0,A,A,0,No Deposit,240.0,NaN,0,Transient,0.193036,0,0


### Missing data, processing agent, company, country data
The below code shows column name of missing data

In [107]:
tmp = pd.to_numeric((traindf.isnull().sum() / traindf.shape[0]).map('{:,.2f}'.format))
tmp[tmp > 0]

country    0.01
agent      0.14
company    0.94
dtype: float64

Probrabaly one booking with `company` is more unlikely to chacel, also the one with specific `agent`, so turn them to boolean

For `Country`, take the nlargest(10)

In [108]:
def Clean_Missing(traindf):
    traindf["children"] = traindf["children"].fillna(0)

    traindf["company"] = (~traindf["company"].isna()).astype("int64")
    traindf["agent"] = (~traindf["agent"].isna()).astype("int64")
    return traindf

    
def Train_Top_Country(traindf):
    TopCountrySet = set(traindf["country"].value_counts()[:10].index)
    traindf["country"] = traindf["country"].apply(lambda x:x if x in TopCountrySet else "Other")
    traindf = pd.get_dummies(traindf, prefix="", prefix_sep="", columns=["country"])
    return traindf, TopCountrySet

traindf = Clean_Missing(traindf)
traindf, TopTenCountry = Train_Top_Country(traindf)

In [109]:
traindf.sample(5)

,hotel,lead_time,arrival_date_year,arrival_date_month,arrival_date_week_number,arrival_date_day_of_month,stays_in_weekend_nights,stays_in_week_nights,adults,children,babies,meal,market_segment,distribution_channel,is_repeated_guest,previous_cancellations,previous_bookings_not_canceled,reserved_room_type,assigned_room_type,booking_changes,deposit_type,agent,company,days_in_waiting_list,customer_type,adr,required_car_parking_spaces,total_of_special_requests,BEL,BRA,DEU,ESP,FRA,GBR,IRL,ITA,NLD,Other,PRT
ID,,,,,,,,,,,,,,,,,,,,,,,,,,,,,,,,,,,,,,,
64099,City Hotel,182,2016,September,40,30,0,2,2,0.0,0,BB,Online TA,TA/TO,0,0,0,A,A,0,No Deposit,1,0,0,Transient,104.833656,0,2,0,0,0,0,0,1,0,0,0,0,0
73637,City Hotel,34,2016,November,48,20,1,0,1,0.0,0,BB,Direct,Direct,0,0,0,A,A,1,No Deposit,1,0,0,Transient,119.162253,0,0,0,0,0,0,0,0,0,0,0,1,0
30746,City Hotel,53,2016,March,12,19,2,3,2,0.0,0,BB,Offline TA/TO,TA/TO,0,0,0,A,A,0,No Deposit,1,0,0,Transient,69.802345,0,1,0,0,1,0,0,0,0,0,0,0,0
53256,Resort Hotel,4,2016,July,31,28,0,3,2,1.0,0,FB,Direct,Direct,0,0,0,D,D,0,No Deposit,0,0,0,Transient,277.593221,0,0,0,0,0,0,0,0,0,0,0,0,1
19920,Resort Hotel,11,2015,December,50,7,1,4,2,0.0,0,BB,Offline TA/TO,TA/TO,0,0,0,A,D,0,No Deposit,1,0,0,Transient,12.817547,0,0,0,0,0,1,0,0,0,0,0,0,0


### Feature Transformation

`children`: `children` + `babies`  
`stays_in_nights`: `stays_in_week_nights` + `stays_in_weekend_nights`    
`hotel`: Convert hotel as `Resort Hotel` to 0, `City Hotel` to 1  

Create new feature `is_same_room`: Check if `reserved_room_type` is same as `assigned_room_type`(If so 1, else 0)

drop: `arrival_date_year` -- While training, most of them are 0  
      `arrival_date_week_number` -- Too trivial  
      `arrival_date_day_of_month` -- Too trivial  

One hot encoding: `meal`  
One hot encoding: `reserved_room_type` and `assigned_room_type`  
One hot encoding: `arrival_date_month`  
One hot encoding: `market_segment` `distribution_channel` `deposit_type` `customer_type`

In [110]:
def Feature_Transform(traindf):
    traindf = pd.get_dummies(traindf, prefix="", prefix_sep="", columns=["arrival_date_month"])
    traindf["stays_in_nights"] = traindf["stays_in_week_nights"] + traindf["stays_in_weekend_nights"] 
    traindf["children"] = traindf["children"] + traindf["babies"]
    traindf = traindf.drop(["babies"], axis=1)
    traindf["hotel"] = traindf["hotel"].map({"Resort Hotel": 0, "City Hotel": 1})

    traindf = traindf.drop(columns=["arrival_date_year", "arrival_date_week_number", "arrival_date_day_of_month"])

    traindf["is_same_room"] = (traindf["reserved_room_type"] == traindf["assigned_room_type"]).map({True: 1, False: 0})
    traindf = pd.get_dummies(traindf, columns=["reserved_room_type", "assigned_room_type"], prefix=["reserved", "assigned"])

    traindf = pd.get_dummies(traindf, columns=["meal", "market_segment", "distribution_channel", "deposit_type", "customer_type"])
    return traindf

traindf = Feature_Transform(traindf)

Final Adjustment:

In [111]:
#traindf = traindf[traindf["adr"] > 10]

In [112]:
traindf.sample(2)

,hotel,lead_time,stays_in_weekend_nights,stays_in_week_nights,adults,children,is_repeated_guest,previous_cancellations,previous_bookings_not_canceled,booking_changes,agent,company,days_in_waiting_list,adr,required_car_parking_spaces,total_of_special_requests,BEL,BRA,DEU,ESP,FRA,GBR,IRL,ITA,NLD,Other,PRT,April,August,December,February,January,July,June,March,May,November,October,September,stays_in_nights,is_same_room,reserved_A,reserved_B,reserved_C,reserved_D,reserved_E,reserved_F,reserved_G,reserved_H,reserved_L,reserved_P,assigned_A,assigned_B,assigned_C,assigned_D,assigned_E,assigned_F,assigned_G,assigned_H,assigned_I,assigned_K,assigned_L,assigned_P,meal_BB,meal_FB,meal_HB,meal_SC,meal_Undefined,market_segment_Aviation,market_segment_Complementary,market_segment_Corporate,market_segment_Direct,market_segment_Groups,market_segment_Offline TA/TO,market_segment_Online TA,market_segment_Undefined,distribution_channel_Corporate,distribution_channel_Direct,distribution_channel_GDS,distribution_channel_TA/TO,distribution_channel_Undefined,deposit_type_No Deposit,deposit_type_Non Refund,deposit_type_Refundable,customer_type_Contract,customer_type_Group,customer_type_Transient,customer_type_Transient-Party
ID,,,,,,,,,,,,,,,,,,,,,,,,,,,,,,,,,,,,,,,,,,,,,,,,,,,,,,,,,,,,,,,,,,,,,,,,,,,,,,,,,,,,,,,,
57065,0,26,0,1,2,0.0,0,0,0,0,1,0,0,246.369970,0,0,0,0,0,0,1,0,0,0,0,0,0,0,1,0,0,0,0,0,0,0,0,0,0,1,1,0,0,0,0,0,1,0,0,0,0,0,0,0,0,0,1,0,0,0,0,0,0,1,0,0,0,0,0,0,0,1,0,0,0,0,0,1,0,0,0,1,0,0,0,0,1,0
41546,1,1,0,1,2,0.0,0,0,0,0,0,0,0,126.661874,0,0,0,0,0,0,0,0,0,1,0,0,0,0,0,0,0,0,0,0,0,1,0,0,0,1,0,1,0,0,0,0,0,0,0,0,0,0,0,0,0,0,1,0,0,0,0,0,0,1,0,0,0,0,0,0,0,1,0,0,0,0,0,1,0,0,0,1,0,0,0,0,1,0


### Train Model and Evaluate best model performance:

Since I only know about regression, I only use SVR as my model, and save the result as 

In [113]:
new_train_x = traindf.drop("adr", axis=1).values
new_train_y = traindf["adr"].values

In [121]:
def HenHenModelSelection():
    regressors = [LinearRegression(),
                   Lasso(),
                   Ridge()
#                    LinearSVR()
                   ]

    from time import time
    for regr in regressors:

        before = time()
        print("="*30)
        print(regr.__class__.__name__)

        X_train, X_test, y_train, y_test = train_test_split(new_train_x, new_train_y, test_size=0.2)
        scaler = StandardScaler()
        scaler = scaler.fit(X_train)
        X_train = scaler.transform(X_train)
        regr.fit(X_train, y_train)
        X_test = scaler.transform(X_test)
        y_pred = regr.predict(X_test)
    
    
        #pipe = make_pipeline(StandardScaler(), regr)
        #crossValScore = cross_validate(pipe, new_train_x, new_train_y, scoring=('r2', 'neg_mean_squared_error'), cv=5) 
        #for metric, value in crossValScore.items():
        #    print(f"{metric}: {np.mean(value):.2f}")
        after = time()

        print(f"MSE: {mean_squared_error(y_test, y_pred)} \nR2_score: {r2_score(y_test, y_pred)}")
        print(f"花費{(after - before):.1f}s")
        
for i in range(10):
    HenHenModelSelection()

LinearRegression
MSE: 931.9904212753119 
R2_score: 0.5894406462820976
花費0.5s
Lasso
MSE: 1027.7638389662454 
R2_score: 0.5628127797513898
花費0.5s
Ridge
MSE: 916.1039058269253 
R2_score: 0.5930255142427332
花費0.3s
LinearRegression
MSE: 1007.3334849139964 
R2_score: 0.5765927407800228
花費0.5s
Lasso
MSE: 2586.4607364179365 
R2_score: 0.33117110085239587
花費0.6s
Ridge
MSE: 953.7330688815024 
R2_score: 0.5897503157135773
花費0.4s
LinearRegression
MSE: 7.372692908290771e+23 
R2_score: -3.174418582353196e+20
花費0.5s
Lasso
MSE: 2591.093563498182 
R2_score: 0.3342128388420287
花費0.5s
Ridge
MSE: 968.3931027917023 
R2_score: 0.5871277684767751
花費0.3s
LinearRegression
MSE: 945.8935056702119 
R2_score: 0.5936198761637153
花費0.5s
Lasso
MSE: 1048.5041102299303 
R2_score: 0.554289846194026
花費0.5s
Ridge
MSE: 967.9572163540936 
R2_score: 0.5913332143472362
花費0.4s
LinearRegression
MSE: 2506.803100806617 
R2_score: 0.3525168152677848
花費0.5s
Lasso
MSE: 1029.9396954858282 
R2_score: 0.5566857725063334
花費0.6s
Ridge
MS

In [123]:
def AaronModelSelection():
    classifiers = [LinearRegression(),
                   ElasticNet(),
                   Lasso(),
                   Ridge(),
                   LinearSVR(),
                   HuberRegressor(),
                   RANSACRegressor(),
    #                TheilSenRegressor(),
                   AdaBoostRegressor(),
    #                BaggingRegressor(),
                   GradientBoostingRegressor(),
                   RandomForestRegressor(),
    #                DecisionTreeRegressor(),
                   RandomForestRegressor(),
                   KNeighborsRegressor(),
                   XGBRegressor()
                   ]

    from time import time
    from sklearn.pipeline import make_pipeline

    for clf in classifiers:
        before = time()
        print("="*30)
        print(clf.__class__.__name__)
        
        pipe = make_pipeline(StandardScaler(), clf)
        crossValScore = cross_validate(pipe, new_train_x, new_train_y, scoring=('r2', 'neg_mean_squared_error'), cv=5) 
        for metric, value in crossValScore.items():
            print(f"{metric}: {abs(np.mean(value)):.2f}")
        after = time()
        print(f"花費{(after - before):.1f}s")

    print("="*30)
    
AaronModelSelection()

LinearRegression
fit_time: 0.44
score_time: 0.02
test_r2: 1029337921015347362660352.00
test_neg_mean_squared_error: 2585045845533193018221264896.00
花費2.3s
ElasticNet
fit_time: 0.40
score_time: 0.02
test_r2: 0.36
test_neg_mean_squared_error: 1612.13
花費2.1s
Lasso
fit_time: 0.46
score_time: 0.02
test_r2: 0.39
test_neg_mean_squared_error: 1550.03
花費2.4s
Ridge
fit_time: 0.26
score_time: 0.02
test_r2: 0.42
test_neg_mean_squared_error: 1475.02
花費1.4s
LinearSVR
fit_time: 13.06
score_time: 0.02
test_r2: 0.40
test_neg_mean_squared_error: 1529.13
花費65.4s
HuberRegressor
fit_time: 2.35
score_time: 0.01
test_r2: 0.41
test_neg_mean_squared_error: 1508.53
花費11.9s
RANSACRegressor
fit_time: 1.66
score_time: 0.02
test_r2: 4444312603124218994884608.00
test_neg_mean_squared_error: 11302239951753383101457235968.00
花費8.4s
AdaBoostRegressor
fit_time: 16.64
score_time: 0.28
test_r2: 1.20
test_neg_mean_squared_error: 5320.15
花費84.7s
GradientBoostingRegressor
fit_time: 30.46
score_time: 0.05
test_r2: 0.51
test_n

KeyboardInterrupt: 

In [99]:
from time import time
regr = GradientBoostingRegressor()

before = time()
print("="*30)
print(regr.__class__.__name__)

X_train, X_test, y_train, y_test = train_test_split(new_train_x, new_train_y, test_size=0.2)
scaler = StandardScaler()
scaler = scaler.fit(X_train)
X_train = scaler.transform(X_train)
regr.fit(X_train, y_train)
X_test = scaler.transform(X_test)
y_pred = regr.predict(X_test)


#pipe = make_pipeline(StandardScaler(), regr)
#crossValScore = cross_validate(pipe, new_train_x, new_train_y, scoring=('r2', 'neg_mean_squared_error'), cv=5) 
#for metric, value in crossValScore.items():
#    print(f"{metric}: {np.mean(value):.2f}")
after = time()

print(f"MSE: {mean_squared_error(y_test, y_pred)} \nR2_score: {r2_score(y_test, y_pred)}")
print(f"花費{(after - before):.1f}s")

GradientBoostingRegressor
MSE: 714.4316749656426 
R2_score: 0.6919131396903766
花費31.1s


In [40]:
regr = GradientBoostingRegressor()
scaler = StandardScaler()
scaler = scaler.fit(new_train_x)
new_train_x = scaler.transform(new_train_x)
regr.fit(new_train_x, new_train_y)

GradientBoostingRegressor()

In [14]:
#for col, value in zip(traindf.drop("adr", axis=1).columns, regr.coef_):
#    print(f"{col}: {value:.2f}")

In [13]:
#X_train, X_test, y_train, y_test = train_test_split(new_train_x, new_train_y, test_size=0.3)
#model = XGBRegressor()
#model = model.fit(X_train, y_train)
#result = permutation_importance(model, X_train, y_train, scoring="r2", n_repeats=5, n_jobs=-1)
#sorted_idx = result.importances_mean.argsort()
#fig, ax = plt.subplots(figsize=(20, 15))

#ax.boxplot(result.importances[sorted_idx].T,
#           vert=False, labels=traindf.drop("adr", axis=1).columns[sorted_idx])
#ax.set_title("Permutation Importance")
#fig.tight_layout()
#plt.show()

In [79]:
testdf = pd.read_csv('../data/test.csv', index_col="ID")

In [80]:
testdf.sample(2)

,hotel,lead_time,arrival_date_year,arrival_date_month,arrival_date_week_number,arrival_date_day_of_month,stays_in_weekend_nights,stays_in_week_nights,adults,children,babies,meal,country,market_segment,distribution_channel,is_repeated_guest,previous_cancellations,previous_bookings_not_canceled,reserved_room_type,assigned_room_type,booking_changes,deposit_type,agent,company,days_in_waiting_list,customer_type,required_car_parking_spaces,total_of_special_requests
ID,,,,,,,,,,,,,,,,,,,,,,,,,,,,
99210,Resort Hotel,67,2017,May,19,10,2,8,3,1.0,0,BB,SWE,Online TA,TA/TO,0,0,0,H,H,2,No Deposit,240.0,NaN,0,Transient,0,1
107051,City Hotel,201,2017,June,25,19,1,1,2,0.0,0,BB,MEX,Online TA,TA/TO,0,0,0,D,D,0,No Deposit,9.0,NaN,0,Transient,0,0


In [59]:
testdf = Clean_Missing(testdf)
testdf = Feature_Transform(testdf)

testdf["country"] = testdf["country"].apply(lambda x:x if x in TopCountrySet else "Other")
testdf = pd.get_dummies(testdf, prefix="", prefix_sep="", columns=["country"])

In [60]:
testdf.sample(2)

,hotel,lead_time,stays_in_weekend_nights,stays_in_week_nights,adults,children,is_repeated_guest,previous_cancellations,previous_bookings_not_canceled,booking_changes,agent,company,days_in_waiting_list,required_car_parking_spaces,total_of_special_requests,April,August,July,June,May,stays_in_nights,is_same_room,reserved_A,reserved_B,reserved_C,reserved_D,reserved_E,reserved_F,reserved_G,reserved_H,reserved_P,assigned_A,assigned_B,assigned_C,assigned_D,assigned_E,assigned_F,assigned_G,assigned_H,assigned_I,assigned_K,assigned_P,meal_BB,meal_FB,meal_HB,meal_SC,meal_Undefined,market_segment_Aviation,market_segment_Complementary,market_segment_Corporate,market_segment_Direct,market_segment_Groups,market_segment_Offline TA/TO,market_segment_Online TA,distribution_channel_Corporate,distribution_channel_Direct,distribution_channel_GDS,distribution_channel_TA/TO,deposit_type_No Deposit,deposit_type_Non Refund,deposit_type_Refundable,customer_type_Contract,customer_type_Group,customer_type_Transient,customer_type_Transient-Party,BEL,BRA,DEU,ESP,FRA,GBR,IRL,ITA,NLD,Other,PRT
ID,,,,,,,,,,,,,,,,,,,,,,,,,,,,,,,,,,,,,,,,,,,,,,,,,,,,,,,,,,,,,,,,,,,,,,,,,,,,
106291,1,320,0,3,2,0.0,0,0,0,0,1,0,0,0,0,0,0,0,1,0,3,1,1,0,0,0,0,0,0,0,0,1,0,0,0,0,0,0,0,0,0,0,1,0,0,0,0,0,0,0,0,1,0,0,0,0,0,1,0,1,0,0,0,1,0,0,0,0,0,0,0,0,0,0,0,1
104870,1,15,0,1,1,0.0,0,0,0,0,1,0,0,0,0,0,0,0,1,0,1,1,1,0,0,0,0,0,0,0,0,1,0,0,0,0,0,0,0,0,0,0,0,0,1,0,0,0,0,0,0,0,0,1,0,0,0,1,1,0,0,0,0,1,0,0,0,0,0,0,0,0,0,0,0,1


In [63]:
len(testdf)

27859

In [64]:
for col in set(traindf.columns) ^ set(testdf.columns):
    testdf[col] = np.zeros(len(testdf))

In [70]:
testdf_x = testdf.drop(["adr"], axis=1)

In [71]:
y_pred = regr.predict(testdf_x)

In [81]:
testdf["adr"] = y_pred

In [82]:
testdf.sample(2)

,hotel,lead_time,arrival_date_year,arrival_date_month,arrival_date_week_number,arrival_date_day_of_month,stays_in_weekend_nights,stays_in_week_nights,adults,children,babies,meal,country,market_segment,distribution_channel,is_repeated_guest,previous_cancellations,previous_bookings_not_canceled,reserved_room_type,assigned_room_type,booking_changes,deposit_type,agent,company,days_in_waiting_list,customer_type,required_car_parking_spaces,total_of_special_requests,adr
ID,,,,,,,,,,,,,,,,,,,,,,,,,,,,,
114433,Resort Hotel,182,2017,July,31,31,1,3,3,0.0,0,HB,PRT,Direct,Direct,0,0,0,G,G,0,No Deposit,NaN,NaN,0,Transient,0,1,31.865461
107499,Resort Hotel,218,2017,June,25,22,0,2,2,0.0,0,BB,CN,Online TA,TA/TO,0,0,0,G,G,1,No Deposit,240.0,NaN,0,Transient,0,2,34.883489


In [83]:
TestAdr = pd.DataFrame(y_pred, columns=["adr"]) 

In [86]:
TestAdr.to_csv("..//data//TestAdr.csv", index=False)

In [137]:
def HenScaler(matrix):
    m, n = matrix.shape
       
    matrix = np.apply_along_axis(lambda x: (x-x.mean())/x.std(), 1, matrix)
    return matrix

In [135]:
a = np.array([[np.random.randint(0, 10) for j in range(5)] for i in range(5)])
a

array([[4, 5, 8, 4, 2],
       [4, 9, 8, 5, 1],
       [5, 6, 1, 1, 5],
       [5, 6, 8, 3, 4],
       [4, 4, 0, 7, 2]])

In [138]:
HenScaler(a)

array([[-0.30618622,  0.20412415,  1.73505523, -0.30618622, -1.32680694],
       [-0.48771311,  1.25411943,  0.90575292, -0.1393466 , -1.53281263],
       [ 0.64993368,  1.11417203, -1.2070197 , -1.2070197 ,  0.64993368],
       [-0.11624764,  0.46499055,  1.62746694, -1.27872403, -0.69748583],
       [ 0.25724788,  0.25724788, -1.45773797,  1.54348727, -0.60024505]])